In [35]:
import json
import pandas as pd
import mappers
import datetime

current_path = 'data/pixie/Indoor/Participant_1/AFE_000_CONFIDENTIAL.json'

df_i_1_AFE = mappers.afe_to_df(current_path)





In [36]:
print(df_i_1_AFE.head())


  afe_left_t  afe_left_ticktime  afe_left_timestamp  afe_left_status  \
0          L            3715918    1699001623169289                0   
1          L            3715935    1699001623185984                0   
2          L            3715952    1699001623202679                0   
3          L            3715968    1699001623219313                0   
4          L            3715985    1699001623236008                0   

   afe_left_counter  afe_left_m_0_0  afe_left_m_0_1  afe_left_m_0_2  \
0             29814           19167            6302            9952   
1             29815           18921            6379           10077   
2             29816           20336            6490           10175   
3             29817           19470            6470           10321   
4             29818           18126            6451           10424   

   afe_left_m_0_3  afe_left_m_0_4  ...  auxSensors_tempEt_v_right_3  \
0           15459           16930  ...                    27.617188  

In [ ]:
first_timestamp = df_i_1_AFE['timestamp'].iloc[0]
date = datetime.strptime(custom_timestamp, '%Y%m%d%H%M%S')
